In [1]:
# importing necessary libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [2]:
# getting data from internet
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text

# using beautiful soup to parse the HTML/XML codes.
soup = BeautifulSoup(raw_wikipedia_page,'xml')
#print(soup.prettify())

In [3]:
# extracting the raw table inside that webpage
table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

# print(table)

# extracting a clean form of the table
for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'): 
        continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

In [4]:
unique_p = set(Postcode)
print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

num of unique Postal codes: 84


In [5]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.to_csv('toronto_part1.csv')
df_toronto.head(14)

,Borough,Neighbourhood,Postcode
0,Scarborough,"Ionview, Kennedy Park",M1K
1,Scarborough,Agincourt,M1S
2,East Toronto,Riverdale,M4K
3,Downtown Toronto,"First Canadian Place, Underground city",M5X
4,North York,"Newtonbrook, Willowdale",M2M
5,Central Toronto,Forest Hill North,M5P
6,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",M1P
7,Etobicoke,Westmount,M9P
8,North York,Hillcrest Village,M2H
9,Central Toronto,Moore Park,M4T


In [6]:
df_toronto.shape

(84, 3)

In [7]:
import geocoder

In [8]:
unique_p = set(Postcode)
# print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

In [9]:
codes=pd.read_csv('Geospatial_Coordinates.csv')
codes.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
#houston_housing=houston_grouped.join(SUM_TotHousing.set_index('SNBNAME'), on='SNBNAME')
#df_toronto.shape
#SUM_TotHousing.rename(columns={'Name':'SNBNAME'},inplace=True)
df_toronto.rename(columns={'Postcode':'Postal Code'},inplace=True)
df_toronto.head()

,Borough,Neighbourhood,Postal Code
0,Scarborough,"Ionview, Kennedy Park",M1K
1,Scarborough,Agincourt,M1S
2,East Toronto,Riverdale,M4K
3,Downtown Toronto,"First Canadian Place, Underground city",M5X
4,North York,"Newtonbrook, Willowdale",M2M


In [11]:
toronto_codes=df_toronto.join(codes.set_index('Postal Code'), on='Postal Code')

In [38]:
toronto_codes.head()


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1K,Scarborough,"Ionview, Kennedy Park",43.727929,-79.262029
1,M1S,Scarborough,Agincourt,43.794200,-79.262029
2,M4K,East Toronto,Riverdale,43.679557,-79.352188
3,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280
4,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493


In [13]:
columnsTitles=['Postal Code','Borough','Neighbourhood','Latitude','Longitude']
toronto_codes=toronto_codes.reindex(columns=columnsTitles)

In [73]:
toronto_codes.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1K,Scarborough,"Ionview, Kennedy Park",43.727929,-79.262029
1,M1S,Scarborough,Agincourt,43.794200,-79.262029
2,M4K,East Toronto,Riverdale,43.679557,-79.352188
3,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280
4,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493


In [26]:
toronto_codes.loc[0, 'Neighbourhood']

'Ionview, Kennedy Park'

In [27]:
neighborhood_latitude = toronto_codes.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_codes.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_codes.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Ionview, Kennedy Park are 43.7279292, -79.26202940000002.


In [74]:
# type your answer here

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 1000 # define radius
 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL
 


'https://api.foursquare.com/v2/venues/explore?&client_id=HKJI0CCM152WSO45MDDDA4AX0ITKZRAIX44U1KU1FPX3BPUM&client_secret=BIBJ3KDBRNNPRRN52FUFOENV4JBFNZWDJJAHMW5LFTU2SNQM&v=20180605&ll=43.7279292,-79.26202940000002&radius=1000&limit=100'

In [75]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c120ecdf594df27e3d05e57'},
 'response': {'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 28,
  'suggestedBounds': {'ne': {'lat': 43.73692920900001,
    'lng': -79.24959814000307},
   'sw': {'lat': 43.71892919099999, 'lng': -79.27446065999696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd381a641b9ef3b646300e6',
       'name': 'Dollarama',
       'location': {'address': '2480 Eglinton Avenue E',
        'crossStreet': 'at Midland Ave',
        'lat': 43.734699781300044,
        'lng': -79.2593157122162,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.734699781300044,
          'lng': -79.2593157122162}],
        'distance': 784,
       

In [76]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [77]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [78]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(15)

,name,categories,lat,lng
0,Dollarama,Discount Store,43.734700,-79.259316
1,Tim Hortons,Coffee Shop,43.734960,-79.257901
2,Chung Moi,Chinese Restaurant,43.732328,-79.269327
3,Giant Tiger,Department Store,43.727447,-79.266240
4,Subway,Sandwich Place,43.731662,-79.268632
5,Tim Hortons,Coffee Shop,43.726895,-79.266157
6,T-Bones Sizzling Steaks & Burgers,Burger Joint,43.735300,-79.256193
7,Tim Hortons,Coffee Shop,43.732134,-79.268264
8,TD Canada Trust,Bank,43.733136,-79.267894
9,KFC,Fast Food Restaurant,43.734838,-79.256080


In [79]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

28 venues were returned by Foursquare.


In [80]:
 Explore Neighborhoods in Toronto

SyntaxError: invalid syntax (<ipython-input-80-91e57f9e445f>, line 1)

In [81]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [82]:
venues = getNearbyVenues(names=toronto_codes['Neighbourhood'],
                                   latitudes=toronto_codes['Latitude'],
                                   longitudes=toronto_codes['Longitude']
                                  )



Ionview, Kennedy Park
Agincourt
Riverdale
First Canadian Place, Underground city
Newtonbrook, Willowdale
Forest Hill North
Dorset Park, Scarborough Town Centre, Wexford Heights
Westmount
Hillcrest Village
Moore Park
Bedford Park
Maryvale, Wexford
CN Tower, King and Spadina, Railway Lands, South Niagara
Rouge, Malvern
Woodbine Gardens, Parkview Hill
Flemingdon Park
Cedarbrae
Lawrence Park
Thorncliffe Park
Henry Farm
Humber Summit
The Beaches
Parkwoods
Design Exchange, Toronto Dominion Centre
Alderwood, Long Branch
Agincourt North, Milliken
Silver Hills, York Mills
Islington Avenue
Parkdale, Roncesvalles
Highland Creek, Rouge Hill, Port Union
Runnymede, Swansea
Toronto Islands, Union Station
Morningside, West Hill
The Annex, Yorkville
Bayview Village
Berczy Park
Deer Park, Rathnelly, South Hill
Church and Wellesley
Mimico NW, The Queensway West, South of Bloor
Markland Wood
High Park
Northwood Park, York University
Commerce Court
Weston
University of Toronto
Emery, Humberlea
Downsview We

In [83]:
print(venues.shape)
venues.head()

(1808, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Ionview, Kennedy Park",43.727929,-79.262029,Giant Tiger,43.727447,-79.266240,Department Store
1,"Ionview, Kennedy Park",43.727929,-79.262029,Tim Hortons,43.726895,-79.266157,Coffee Shop
2,"Ionview, Kennedy Park",43.727929,-79.262029,Hakka No.1,43.727688,-79.266057,Chinese Restaurant
3,"Ionview, Kennedy Park",43.727929,-79.262029,Dollarama,43.726904,-79.265886,Discount Store
4,"Ionview, Kennedy Park",43.727929,-79.262029,Tandy Leather,43.726974,-79.266513,Hobby Shop


In [84]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, Milliken",2,2,2,2,2,2
"Alderwood, Long Branch",10,10,10,10,10,10
"Bathurst Manor, Wilson Heights",17,17,17,17,17,17
Bayview Village,4,4,4,4,4,4
"Beaumond Heights, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",11,11,11,11,11,11
Bedford Park,24,24,24,24,24,24
Berczy Park,55,55,55,55,55,55


In [85]:
print('There are {} uniques categories.'.format(len(venues['Venue Category'].unique())))

There are 262 uniques categories.


In [86]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] =venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [87]:
onehot.shape

(1808, 262)

In [88]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide, King",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.010000,0.000000,0.000000,0.000000,0.0000,0.010000,0.000000,0.000000,0.01
1,Agincourt,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.00
2,"Agincourt North, Milliken",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.00
3,"Alderwood, Long Branch",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.00
4,"Bathurst Manor, Wilson Heights",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.058824,0.000000,0.0000,0.000000,0.000000,0.000000,0.00
5,Bayview Village,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.00
6,"Beaumond Heights, Jamestown, Mount Olive, Silv...",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.00
7,Bedford Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.00
8,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.00
9,Birch Cliff,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.00


In [89]:
grouped.shape

(82, 262)

In [90]:
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2  American Restaurant  0.04
3      Thai Restaurant  0.04
4           Steakhouse  0.04


----Agincourt----
                venue  freq
0      Sandwich Place  0.25
1      Breakfast Spot  0.25
2              Lounge  0.25
3        Skating Rink  0.25
4  Miscellaneous Shop  0.00


----Agincourt North, Milliken----
                      venue  freq
0                Playground   0.5
1                      Park   0.5
2             Moving Target   0.0
3            Medical Center   0.0
4  Mediterranean Restaurant   0.0


----Alderwood, Long Branch----
          venue  freq
0   Pizza Place   0.2
1      Pharmacy   0.1
2   Coffee Shop   0.1
3  Dance Studio   0.1
4           Pub   0.1


----Bathurst Manor, Wilson Heights----
              venue  freq
0       Coffee Shop  0.12
1     Shopping Mall  0.06
2  Sushi Restaurant  0.06
3    Sandwich Place  0.06
4         Gift Shop  0.06


----Bayview

                venue  freq
0            Bus Line   0.2
1  Dim Sum Restaurant   0.2
2                Park   0.2
3         Swim School   0.2
4                Lake   0.2


----Leaside----
                 venue  freq
0          Coffee Shop  0.09
1  Sporting Goods Shop  0.09
2         Burger Joint  0.06
3        Grocery Store  0.06
4            Pet Store  0.03


----Little Portugal, Trinity----
         venue  freq
0          Bar  0.12
1         Café  0.05
2  Men's Store  0.05
3  Coffee Shop  0.05
4   Restaurant  0.05


----Maple Leaf Park----
                        venue  freq
0            Basketball Court  0.25
1  Construction & Landscaping  0.25
2                      Bakery  0.25
3                        Park  0.25
4                 Yoga Studio  0.00


----Markland Wood----
         venue  freq
0  Pizza Place  0.14
1         Café  0.14
2     Pharmacy  0.14
3   Beer Store  0.14
4         Park  0.14


----Maryvale, Wexford----
                           venue  freq
0      Middle Easter

In [91]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [92]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King",Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Hotel,Bar,Clothing Store,Asian Restaurant,Gym
1,Agincourt,Breakfast Spot,Lounge,Sandwich Place,Skating Rink,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store
2,"Agincourt North, Milliken",Playground,Park,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
3,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Gym,Pharmacy,Skating Rink,Sandwich Place,Dance Studio,Pool,Pub,Diner
4,"Bathurst Manor, Wilson Heights",Coffee Shop,Frozen Yogurt Shop,Pizza Place,Deli / Bodega,Restaurant,Diner,Sandwich Place,Bridal Shop,Shopping Mall,Fast Food Restaurant
5,Bayview Village,Japanese Restaurant,Café,Bank,Chinese Restaurant,Women's Store,Diner,Dog Run,Doner Restaurant,Donut Shop,Drugstore
6,"Beaumond Heights, Jamestown, Mount Olive, Silv...",Grocery Store,Pharmacy,Beer Store,Japanese Restaurant,Sandwich Place,Discount Store,Fried Chicken Joint,Fast Food Restaurant,Coffee Shop,Pizza Place
7,Bedford Park,Coffee Shop,Fast Food Restaurant,Italian Restaurant,Greek Restaurant,Pub,Liquor Store,Comfort Food Restaurant,Sandwich Place,Juice Bar,Restaurant
8,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Seafood Restaurant,Steakhouse,Italian Restaurant,Farmers Market,Bakery,Pub,Café
9,Birch Cliff,College Stadium,General Entertainment,Café,Skating Rink,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


In [94]:
from sklearn.cluster import KMeans

In [95]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 3, 0, 1, 1, 3, 1, 1, 1, 3])

In [96]:
toronto_codes.head(2)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1K,Scarborough,"Ionview, Kennedy Park",43.727929,-79.262029
1,M1S,Scarborough,Agincourt,43.794200,-79.262029


In [97]:
neighborhoods_venues_sorted.head(2)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King",Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Hotel,Bar,Clothing Store,Asian Restaurant,Gym
1,Agincourt,Breakfast Spot,Lounge,Sandwich Place,Skating Rink,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store


In [98]:
toronto_codes.rename(index=str, columns={'Neighbourhood':'Neighborhood'})
toronto_codes.head(2)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1K,Scarborough,"Ionview, Kennedy Park",43.727929,-79.262029
1,M1S,Scarborough,Agincourt,43.794200,-79.262029


In [114]:
neighborhoods_venues_sorted.shape

(82, 11)

In [116]:
toronto_codes
toronto=toronto_codes[:82]
toronto.shape

(82, 5)

In [118]:
merged = toronto

# add clustering labels
merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

merged.head() # check the last columns!

C:\ZU_Program_Files\anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1K,Scarborough,"Ionview, Kennedy Park",43.727929,-79.262029,1,Discount Store,Department Store,Hobby Shop,Chinese Restaurant,Train Station,Coffee Shop,Bus Station,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
1,M1S,Scarborough,Agincourt,43.794200,-79.262029,3,Breakfast Spot,Lounge,Sandwich Place,Skating Rink,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store
2,M4K,East Toronto,Riverdale,43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Cosmetics Shop,Brewery,Bubble Tea Shop,Restaurant,Café
3,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280,1,Coffee Shop,Café,Hotel,Restaurant,Steakhouse,American Restaurant,Asian Restaurant,Seafood Restaurant,Deli / Bodega,Gastropub
4,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493,1,Park,Women's Store,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [125]:
import folium # map rendering library
import matplotlib.cm as cm
import matplotlib.colors as colors


In [126]:
latitude = 43.7
longitude = -79.4

In [128]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Neighbourhood'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [129]:
merged.loc[merged['Cluster Labels'] == 0, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,East Toronto,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Cosmetics Shop,Brewery,Bubble Tea Shop,Restaurant,Café
24,Etobicoke,0,Pizza Place,Coffee Shop,Gym,Pharmacy,Skating Rink,Sandwich Place,Dance Studio,Pool,Pub,Diner
54,Etobicoke,0,Baseball Field,Women's Store,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
60,East York,0,Coffee Shop,Sporting Goods Shop,Grocery Store,Burger Joint,Breakfast Spot,Supermarket,Fish & Chips Shop,Beer Store,Bike Shop,Liquor Store
65,East York,0,Park,Convenience Store,Women's Store,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Electronics Store
71,Etobicoke,0,River,Park,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
77,Scarborough,0,Motel,American Restaurant,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Women's Store,Dim Sum Restaurant


In [130]:
merged.loc[merged['Cluster Labels'] == 1, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,1,Discount Store,Department Store,Hobby Shop,Chinese Restaurant,Train Station,Coffee Shop,Bus Station,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
3,Downtown Toronto,1,Coffee Shop,Café,Hotel,Restaurant,Steakhouse,American Restaurant,Asian Restaurant,Seafood Restaurant,Deli / Bodega,Gastropub
4,North York,1,Park,Women's Store,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
6,Scarborough,1,Indian Restaurant,Vietnamese Restaurant,Latin American Restaurant,Chinese Restaurant,Pet Store,Comic Shop,Concert Hall,Event Space,Ethiopian Restaurant,Empanada Restaurant
7,Etobicoke,1,Pizza Place,Intersection,Middle Eastern Restaurant,Coffee Shop,Sandwich Place,Chinese Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run
8,North York,1,Golf Course,Mediterranean Restaurant,Dog Run,Pool,Drugstore,Diner,Discount Store,Doner Restaurant,Donut Shop,Women's Store
12,Downtown Toronto,1,Airport Lounge,Airport Service,Airport Terminal,Boutique,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Harbor / Marina,Plane
14,East York,1,Fast Food Restaurant,Pizza Place,Gastropub,Pharmacy,Rock Climbing Spot,Bank,Intersection,Athletics & Sports,Pet Store,Gym / Fitness Center
15,North York,1,Coffee Shop,Gym,Beer Store,Asian Restaurant,Chinese Restaurant,Fast Food Restaurant,Bike Shop,Sporting Goods Shop,Sandwich Place,Japanese Restaurant
17,Central Toronto,1,Dim Sum Restaurant,Lake,Swim School,Bus Line,Park,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Diner,Dumpling Restaurant


In [131]:
merged.loc[merged['Cluster Labels'] == 2, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Scarborough,2,Playground,Park,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
36,Central Toronto,2,Coffee Shop,Pub,American Restaurant,Convenience Store,Bagel Shop,Supermarket,Fried Chicken Joint,Sports Bar,Sushi Restaurant,Pizza Place


In [132]:
merged.loc[merged['Cluster Labels'] == 3, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,3,Breakfast Spot,Lounge,Sandwich Place,Skating Rink,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store
5,Central Toronto,3,Trail,Park,Sushi Restaurant,Jewelry Store,Women's Store,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
9,Central Toronto,3,Trail,Tennis Court,Gym,Women's Store,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
10,North York,3,Coffee Shop,Fast Food Restaurant,Italian Restaurant,Greek Restaurant,Pub,Liquor Store,Comfort Food Restaurant,Sandwich Place,Juice Bar,Restaurant
11,Scarborough,3,Middle Eastern Restaurant,Auto Garage,Paper / Office Supplies Store,Smoke Shop,Breakfast Spot,Shopping Mall,Sandwich Place,Donut Shop,Discount Store,Dog Run
13,Scarborough,3,Fast Food Restaurant,Women's Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
16,Scarborough,3,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Fried Chicken Joint,Thai Restaurant,Caribbean Restaurant,Donut Shop,Dog Run,Doner Restaurant
22,North York,3,Food & Drink Shop,Park,Fast Food Restaurant,Event Space,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Diner,Eastern European Restaurant
23,Downtown Toronto,3,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gastropub,Italian Restaurant,Gym,Deli / Bodega,Bar
29,Scarborough,3,History Museum,Bar,Women's Store,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store


In [133]:
merged.loc[merged['Cluster Labels'] == 4, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
40,West Toronto,4,Café,Grocery Store,Bar,Mexican Restaurant,Fried Chicken Joint,Diner,Cajun / Creole Restaurant,Bakery,Speakeasy,Furniture / Home Store
